## Description
The first part is focused on inferring the missing value about skin color rating using machine learning algorithm. The second part try to infer wether referees' behavior change depending of the skin-color of the player.

Let start by doing some imports.

In [88]:
import os
import pandas as pd
import numpy as np
import sklearn

%matplotlib inline
import matplotlib.pyplot as plt                         # Matplotlib's pyplot: MATLAB-like syntax
import seaborn as sns                                   # For pretty plots

In [111]:
df = pd.read_csv(os.path.join('data', 'CrowdstormingDataJuly1st.csv'))
dfd = pd.read_csv(os.path.join('data', 'crowdstorm_disaggregated.csv'))
df_cols = set(df.columns)
dfd_cols = set(dfd.columns)
df['skintone'] = 0.5 * (df['rater1']+df['rater2'])
df_skinton = df[["playerShort", "skintone"]].reset_index().groupby('playerShort', axis=0).mean()
mean_skin = set(df_skinton['skintone'].dropna().unique().tolist())
unique_skin = set(df['skintone'].dropna().unique().tolist())
LABEL_FIELDS = ['club', 'leagueCountry', 'position', 'Alpha_3']
UNKNOWN_LABEL = 'MISSING'
label_encoders = {}

from sklearn.preprocessing import LabelEncoder

df_categorized = df.copy()

for label in LABEL_FIELDS:
    encoder = LabelEncoder()
    values = df_categorized[label].fillna(UNKNOWN_LABEL, inplace=False)
    encoder.fit(values)
    transformed = encoder.transform(values)
    df_categorized[label] = transformed
    
    label_encoders[label] = encoder
    
# Special case for the skintone where we need to keep NaN values
encoder = LabelEncoder()
skintone = df_categorized['skintone']
skintone = skintone[skintone.notnull()]
encoder.fit(skintone)
df_categorized.loc[skintone.index, 'skintone'] = encoder.transform(skintone)
label_encoders['skintone'] = encoder


df_subset = df_categorized[['playerShort', 'club', 'leagueCountry', 'position','games', 'yellowCards', 'redCards', 'meanIAT', 'skintone', 'birthday', 'height',
       'weight', 'victories', 'ties', 'defeats', 'goals']].copy()

for f in ['meanIAT', 'yellowCards', 'redCards']:
    df_subset[f].fillna(0, inplace=True)
df_subset["skintone"] = df["skintone"]
df_subset["cost"] = df_subset['meanIAT']*(df_subset['yellowCards'] + 3*df_subset['redCards'])

# Unsupervised clustering

We keep the dataframe `df_subset` because we want to keep the `cost` column from part 1 as a likely useful aggregate of racial bias that a player received.



In [ ]:
df_aggregated = df_subset.drop(["meanIAT", "club", "leagueCountry"], 1).set_index(["playerShort"]).groupby(level=0).agg(
    {
        "skintone": "mean",
        "height": "mean",
        "weight": "mean",
        "position": "first",
        "games": "max",
        "yellowCards": "sum",
        "redCards": "sum",
        "cost": "sum",
        "victories": "max",
        "ties": "max",
        "defeats": "max",
        "goals": "max",
     }
).reset_index().drop(["playerShort"], 1).dropna()
df_aggregated["yellowCards"] = df_aggregated["yellowCards"] / df_aggregated["games"]
df_aggregated["redCards"] = df_aggregated["redCards"] / df_aggregated["games"]
df_aggregated["cost"] = df_aggregated["cost"] / df_aggregated["games"]

x = df_aggregated.drop(["games", "skintone"], axis=1)

data = sklearn.preprocessing.scale(x.as_matrix())
features = list(x.columns)

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

kmeans = KMeans(n_clusters=2, random_state=0).fit(data)

We arbitrarily split the skintone feature in two parts, the ones strictly below 0.5 and the ones equal or above 0.5.

In [128]:
def accuracy(skintone, labels):
    skintone = skintone < 0.5
    score = ((skintone - labels).apply(np.abs)).mean()
    if score < 0.5:
        return 1 - score
    return score

x["label"] = kmeans.labels_
cur_score = silhouette_score(data, kmeans.labels_)
acc = accuracy(df_aggregated["skintone"], kmeans.labels_)

print("Original silhouette score: {} \t accuracy: {}".format(cur_score, acc))

print(features)
for i in range(0, len(features)):
    score_changed = False
    for f in features:
        data = sklearn.preprocessing.scale(x.drop(f, 1).as_matrix())
        kmeans = KMeans(n_clusters=2, random_state=0).fit(data)
        score = silhouette_score(data, kmeans.labels_)
        acc = accuracy(df_aggregated["skintone"], kmeans.labels_)
        if score > cur_score:
            print("Dropping {} gives Silhouette score {} \t accuracy: {}".format(f, score, acc))
            #print(pd.concat([df_aggregated.reset_index()["skintone"], pd.Series(kmeans.labels_)], axis=1))
            del(features[features.index(f)])
            cur_score = score
            x = x.drop(f, 1)
            break
    

0     0.125
1     0.125
2     0.250
3     0.000
4     0.250
7     0.750
9     0.875
11    0.875
13    1.000
14    0.875
15    1.000
16    0.375
18    0.000
19    0.750
21    0.250
23    0.000
24    0.000
25    0.000
29    0.000
30    0.000
Name: skintone, dtype: float64
Original silhouette score: 1.0 	 accuracy: 0.5287723785166241
[]
